In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import numpy as np
import cv2

app = Flask(__name__)
model = load_model('path/to/save/your/model')

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    image = cv2.imdecode(np.fromstring(file.read(), np.uint8), cv2.IMREAD_COLOR)
    image = cv2.resize(image, (256, 256))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    
    restored_image = model.predict(image)
    restored_image = np.squeeze(restored_image)
    restored_image = restored_image * 255.0
    restored_image = restored_image.astype(np.uint8)

    _, buffer = cv2.imencode('.jpg', restored_image)
    restored_image_bytes = buffer.tobytes()

    response = {
        'restored_image': restored_image_bytes
    }
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True)